In [1]:
import dimod
from dimod import Integer, Binary
import networkx as nx
from matplotlib import pyplot as plt
import random

In [2]:
foods = {
    'rice': {'Calories': 100, 'Protein': 3, 'Fat': 1, 'Carbs': 22, 'Fiber': 2, 'Taste': 7, 'Cost': 2.5, 'Units':'continuous'},
    'tofu': {'Calories': 140, 'Protein': 17, 'Fat': 9, 'Carbs': 3, 'Fiber': 2,
           'Taste': 2, 'Cost': 4.0, 'Units': 'continuous'},
  'banana': {'Calories': 90, 'Protein': 1, 'Fat': 0, 'Carbs': 23, 'Fiber': 3,
             'Taste': 10, 'Cost': 1.0, 'Units': 'discrete'},
  'lentils': {'Calories': 150, 'Protein': 9, 'Fat': 0, 'Carbs': 25, 'Fiber': 4,
              'Taste': 3, 'Cost': 1.3, 'Units': 'continuous'},
  'bread': {'Calories': 270, 'Protein': 9, 'Fat': 3, 'Carbs': 50, 'Fiber': 3,
            'Taste': 5, 'Cost': 0.25, 'Units': 'continuous'},
  'avocado': {'Calories': 300, 'Protein': 4, 'Fat': 30, 'Carbs': 20, 'Fiber': 14,
              'Taste': 5, 'Cost': 2.0, 'Units': 'discrete'}}

In [3]:
min_nutrients = {'Protein': 50, 'Fat': 30, 'Carbs': 130, 'Fiber': 30}
max_calories = 2000

### Variables
Instantiating some real and integer variables in a list `quantities`, that in the solutions will be assigned values for the selected quantities of every availbale food. 

In [4]:
quantities = [dimod.Real(f"{food}") if foods[food]['Units'] == 'continuous' 
                                    else dimod.Integer(f"{food}")
                                    for food in foods.keys()]

such 'variables; are quadratic model objects

In [18]:
print(quantities[0])

QuadraticModel({'rice': 1.0}, {}, 0.0, {'rice': 'REAL'}, dtype='float64')


In [6]:
print(quantities[2])

QuadraticModel({'banana': 1.0}, {}, 0.0, {'banana': 'INTEGER'}, dtype='float64')


Bounds on the range of values for non-binary variables shrink the solution space the solver must search, so it is helpful to set such bounds; for many problems, you can find bounds from your knowledge of the problem. In this case, no food by itself should be assigned a quantity that exceeds max_calories.

In [7]:
for index, food in enumerate(foods.keys()):
    upper_bound = max_calories / foods[food]["Calories"]
    quantities[index].set_upper_bound(food, upper_bound)

In [8]:
quantities[0].upper_bound('rice')

20.0

Lower bounds are actually required in this problem for correct formulation: a valid mathematical solution might be to offset the calories of gorging on large numbers of tasty bananas by eating a negative amount of high-in-calories bread, so the formulation must include the impossibility of consuming negative quantities of food. Because Ocean sets a default value of zero for ~dimod.quadratic.Real variables, no explicit configuration is needed.

## Objective Function

The objective function must maximize taste of the diet’s foods while minimizing purchase cost.

To maximize taste and minimize cost is to assign values to the variables that represent quantities of each food, 
, such that when multiplied by coefficients representing the cost, 
, or taste, 
, of each food, form the linear terms of the following summations to be optimized:


Instantiate the CQM

In [9]:
cqm = dimod.ConstrainedQuadraticModel()

define `total_mix` to calculate the summations for any given category such as calories

In [10]:
def total_mix(quantity, category):
    return sum(q * c for q, c, in zip(quantity, (foods[food][category] for food in foods.keys())))

Set the objective. because Ocean solvers minimize objectives to maximize taste `Taste` is multiplied by -1

In [11]:
cqm.set_objective(-total_mix(quantities, 'Taste') + 6*total_mix(quantities, 'Cost'))

Here is $\alpha$ is chosen as -1 and $\beta$ is chosen as 6 through tuning the solution 

## Constraints
The problem has the following constraints:
1. Calories: no more than 2000
2. Protein: at least 50
3. Fat: at least 30
4. Carbs: at least 130
5. Fiber: at least 30

In [12]:
cqm.add_constraint(total_mix(quantities, 'Calories') <= max_calories, label='Calories')

'Calories'

Require that the daily minimum of each nutrient is met or exceeded

In [13]:
for nutrient, amount in min_nutrients.items():
    cqm.add_constraint(total_mix(quantities, nutrient) >= amount, label=nutrient)

These constraints can be accessed as a dictionary with the labels as keys

In [14]:
list(cqm.constraints.keys())

['Calories', 'Protein', 'Fat', 'Carbs', 'Fiber']

In [15]:
print(cqm.constraints['Calories'].to_polystring())

100*rice + 140*tofu + 90*banana + 150*lentils + 270*bread + 300*avocado <= 2000.0


In [16]:
print(cqm.constraints['Protein'].to_polystring())

3*rice + 17*tofu + banana + 9*lentils + 9*bread + 4*avocado >= 50.0


### Solve the Problem by Solving

In [19]:
from dwave.system import LeapHybridCQMSampler
sampler = LeapHybridCQMSampler()

In [20]:
sampleset = sampler.sample_cqm(cqm)
feasible_sampleset = sampleset.filter(lambda row: row.is_feasible)
print("{} feasible solutions of {}.".format(len(feasible_sampleset), len(sampleset)))

72 feasible solutions of 112.


Defining a utility function, `print_diet` to print the returned solujtions in an intuitive format

In [21]:
def print_diet(sample):
    diet = {food: round(quantity, 1) for food, quantity in sample.items()}
    print(f"Diet: {diet}")
    taste_total = sum(foods[food]['Taste'] * amount for food, amount in sample.items())
    cost_total = sum(foods[food]['Cost'] * amount for food, amount in sample.items())
    print(f"Total Taste of {round(taste_total, 2)} at Cost ${round(cost_total, 2)}")
    for constraint in cqm.iter_constraint_data(sample):
        print(f"{constraint.label} (nominal: {constraint.rhs_energy}) : {round(constraint.lhs_energy)}")

In [22]:
best = feasible_sampleset.first.sample
print_diet(best)

Diet: {'avocado': 1.0, 'banana': 6.0, 'bread': 4.1, 'lentils': 0.3, 'rice': 0.0, 'tofu': 0.0}
Total Taste of 86.56 at Cost $9.46
Calories (nominal: 2000.0) : 2000
Protein (nominal: 50.0) : 50
Fat (nominal: 30.0) : 42
Carbs (nominal: 130.0) : 372
Fiber (nominal: 30.0) : 46
